# Data Cleaning

After mapping the data points using raw data in Tableau, we observed that the raw data have following issues:

- In 'animal_data' sheet, data points where 'shelter_id' is 'San Diego' are noisy:

 a) 'intake_date' ranges from 2021-2029, which doesnt make sense.
 
 b) species, longitude, latitude, distances are empty, we are only given the found and outcome address. Also, 'found addresses' have issues (e.g. '*Unknown finder did not give information, Escondido CA 92026*'). Inaccurate address leads to inaccurate distance calculations. 
 
 c) Shelter 'San Diego' is located in the San Diego county in Texas, but most addresses in 'animal_data' are in San Diego, CA, which is really far from Texas. It is unclear why this is the case considering there are 3 shelters in California.

>**We decide to exclude san diego data. Without knowing the species, these data can't help us to build models to predict animal behavior.**
       
- After plotting out animal travel paths, paths seems unreasonable for data in ‘Mcallen’ shelter. For example, many data points have distance traveled over 400 miles (with a dozen cases over 1000 miles), which does not seem very feasible.

>**We decide to recalculate longitude, latitude, and distance for all data in Mcallen.**
        


In [71]:
import pandas as pd
import numpy as np
from geopy.geocoders import ArcGIS
from geopy import distance

In [31]:
df = pd.read_excel("ampa_wmt_rto_hackathon_july-22_data.xlsx", sheet_name=['animal_data','shelter_data'])
animal_data = df['animal_data']
shelter_data = df['shelter_data']

In [38]:
animal_data.head()

,shelter_id,intake_date,Species,found_lng,found_lat,outcome_lng,outcome_lat,distance_miles,found_address,outcome_address
0,Rochester,2021-01-28,Dog,-77.633991,43.177237,-77.634618,43.178387,0.085445,158 Glenwood ave Rochester 14613,54 Tacoma Street Rochester NY 14613
1,Rochester,2021-01-30,Dog,-77.599173,43.185495,-77.583367,43.171650,1.245466,1094 HUDSON AVE 14621 Rochester 14621,31 CROMBIE ST Rochester NY 14605
2,Rochester,2021-01-30,Dog,-77.617074,43.241926,-77.615373,43.249483,0.528722,17 Hughes pl Rochester 14608,4313 LAKE AVE Rochester NY 14612
3,Rochester,2021-02-01,Dog,-77.643231,43.180933,-77.645970,43.179665,0.163735,DRIVING PK and 2 CANARY ST. Rochester 14608,1 CANARY ST Rochester NY 14613
4,Rochester,2021-02-03,Dog,-77.650684,43.143780,-77.653187,43.145435,0.170467,74 Thurston Rd Rochester 14608,216 DEPEW Street Rochester NY 14611


In [37]:
animal_data.describe()

,found_lng,found_lat,outcome_lng,outcome_lat,distance_miles
count,19966.000000,19966.000000,19966.000000,19966.000000,23054.000000
mean,-102.310446,33.293523,-102.317015,33.286317,7.352016
std,11.093117,2.598450,11.076127,2.570052,92.484657
min,-122.299066,-22.906847,-122.456421,25.927541,0.000000
25%,-111.033083,32.279603,-111.037122,32.280073,0.102528
50%,-96.897800,32.740239,-96.906801,32.739913,0.412854
75%,-96.714288,33.840206,-96.712254,33.843464,2.305908
max,-43.172897,53.933271,-77.173657,43.319909,4998.633934


In [34]:
animal_data[animal_data['shelter_id'] == 'San Diego'].describe()

,found_lng,found_lat,outcome_lng,outcome_lat,distance_miles
count,0.0,0.0,0.0,0.0,3088.000000
mean,NaN,NaN,NaN,NaN,27.276595
std,NaN,NaN,NaN,NaN,209.974131
min,NaN,NaN,NaN,NaN,0.000000
25%,NaN,NaN,NaN,NaN,0.196353
50%,NaN,NaN,NaN,NaN,0.591235
75%,NaN,NaN,NaN,NaN,2.505213
max,NaN,NaN,NaN,NaN,2963.828729


In [35]:
mcallen_df = animal_data[animal_data['shelter_id'] == 'Mcallen']
mcallen_df.head()

,shelter_id,intake_date,Species,found_lng,found_lat,outcome_lng,outcome_lat,distance_miles,found_address,outcome_address
855,Mcallen,2017-01-01,Dog,-98.261393,26.162872,-98.267644,26.163505,0.390768,NORMA AVE. Texas 78503 United States,3521 NORMA Avenue Mcallen TX 78503
856,Mcallen,2017-01-01,Dog,-98.207756,26.262344,-98.263931,26.209803,5.024621,6808 n pecking mcallen Texas 78504 United States,202 N 40TH Mcallen TX 78501
857,Mcallen,2017-01-01,Dog,-98.077289,26.166055,-98.240971,26.147963,10.245089,808 S. VALLEY VIEW RD. Texas 78537 United States,6404 S 11th Street Mcallen TX 78503
858,Mcallen,2017-01-01,Dog,-98.156419,27.205221,-98.105564,26.360682,58.229686,Falfurias Check Point Texas 78504 United States,3901 ENERGY AVE. Edinburg TX 78542
859,Mcallen,2017-01-01,Dog,-98.192988,26.313007,-98.186176,26.279896,2.318307,Shunior rd. edinburg Texas 78541 United States,2622 Garza Street Edinburg TX 78539


In [36]:
mcallen_df.describe()

,found_lng,found_lat,outcome_lng,outcome_lat,distance_miles
count,751.000000,751.000000,751.000000,751.000000,751.000000
mean,-98.029965,26.559832,-98.178822,26.366929,52.046231
std,3.025470,2.745142,1.146668,1.002667,276.635945
min,-122.030796,-22.906847,-121.666497,25.927541,0.000000
25%,-98.235875,26.219294,-98.240994,26.215299,0.127267
50%,-98.197484,26.258901,-98.204307,26.252357,1.203821
75%,-98.149810,26.294320,-98.151966,26.291066,5.166788
max,-43.172897,53.933271,-80.432221,43.066712,4998.633934


## Test Example

We first test out the data point where 'distance_mile' is 4998.63. 'distance_miles' in raw data is 4998.63, and
our calculated distance is 0.07 miles. We also input the entire address in google map, google map cannot find the address using the given zipcodes. It auto-corrected itself and calculated the route using the street addresses in 78539. We suspect there might be data-entry problem in zipcodes.

In [72]:
#test example
nom = ArcGIS()
start = nom.geocode("1615 Rio De Janeiro Texas 78542 United States")
destination = nom.geocode("1624 RIO DE JANEIRO Street Edinburg TX 78539")

start_lat = start.latitude
start_long = start.longitude
destination_lat = destination.latitude
destination_long = destination.longitude

start_latln = start[-1]
dest_latln = destination[-1]
dist = distance.distance(start_latln, dest_latln).miles
print(dist)

0.07546805222069013


In [42]:
start = nom.geocode("1615 Rio De Janeiro Texas 78542 United States")
start[-1]

(26.268864284379205, -98.18707978104631)

In [65]:
mcallen_df['found_address'].iloc[0]

'NORMA AVE. Texas 78503 United States'

In [73]:
mcallen_df['found_address']
# rewrite found and outcome longitudes and latitudes
for i in range(len(mcallen_df)):
    start = nom.geocode(mcallen_df['found_address'].iloc[i])
    destination = nom.geocode(mcallen_df['outcome_address'].iloc[i])

    mcallen_df['found_lat'].iloc[i] = start.latitude
    mcallen_df['found_lng'].iloc[i] = start.longitude
    mcallen_df['outcome_lat'].iloc[i] = destination.latitude
    mcallen_df['outcome_lng'].iloc[i]  = destination.longitude

    start_latln = start[-1]
    dest_latln = destination[-1]
    mcallen_df['distance_miles'].iloc[i]  = distance.distance(start_latln, dest_latln).miles

In [74]:
mcallen_df.describe()

,found_lng,found_lat,outcome_lng,outcome_lat,distance_miles
count,751.000000,751.000000,751.000000,751.000000,751.000000
mean,-98.173274,26.445932,-98.178510,26.348447,25.730791
std,0.585904,1.204889,1.145596,0.957362,124.535747
min,-104.345450,26.089256,-121.666351,25.927514,0.000000
25%,-98.236405,26.215770,-98.240993,26.215118,0.140307
50%,-98.197994,26.252600,-98.204324,26.252236,1.478609
75%,-98.151918,26.294325,-98.151763,26.291078,4.908854
max,-89.560960,45.075610,-80.431990,43.066742,1556.412396


After recalculating all longitudes and latitudes, our new Mcallen data has maximum distance of 1556 miles and an average of 25.73 miles. There are 4 data points with distance traveled more than 1000 miles and 33 instances of distance traveled more than 100 miles. All of these are lower than raw data, and likely reflect data more relevent to reality.

In [96]:
print(f"Number of records with distance traveled over 1000 miles: {sum(mcallen_df['distance_miles'] > 1000)}")
mcallen_df[mcallen_df['distance_miles'] > 1000]

Number of records with distance traveled over 1000 miles: 4


,shelter_id,intake_date,Species,found_lng,found_lat,outcome_lng,outcome_lat,distance_miles,found_address,outcome_address
862,Mcallen,2017-01-01,Dog,-98.215634,26.267603,-121.666351,36.796903,1556.412396,trenton and bentsen Texas 78504 United States,8305 Prunedale N Road Salinas CA 93907
879,Mcallen,2017-01-01,Dog,-98.138603,32.445356,-80.431990,37.118714,1055.412471,30267 N. HWY 281 Texas 24073 United States,5407 THADDEUS LN Christiansburg VA 24073
1214,Mcallen,2017-01-01,Dog,-89.560960,45.075610,-98.170698,26.187266,1387.475814,10th - wisconsin Texas 78501 United States,1300 E Jones Pharr TX 78577
1560,Mcallen,2017-01-01,Dog,-98.338858,26.282932,-88.370899,43.066742,1286.422371,801 inspiration rd Texas 78574 United States,1329 Parry Lane Hartland WI 53029


In [101]:
print(f"Number of records with distance traveled over 100 miles: {sum(mcallen_df['distance_miles'] > 100)}")

mcallen_df[mcallen_df['distance_miles'] > 100]

Number of records with distance traveled over 100 miles: 33


,shelter_id,intake_date,Species,found_lng,found_lat,outcome_lng,outcome_lat,distance_miles,found_address,outcome_address
862,Mcallen,2017-01-01,Dog,-98.215634,26.267603,-121.666351,36.796903,1556.412396,trenton and bentsen Texas 78504 United States,8305 Prunedale N Road Salinas CA 93907
879,Mcallen,2017-01-01,Dog,-98.138603,32.445356,-80.431990,37.118714,1055.412471,30267 N. HWY 281 Texas 24073 United States,5407 THADDEUS LN Christiansburg VA 24073
887,Mcallen,2017-01-01,Dog,-94.179700,31.796160,-98.193605,26.335073,447.609404,PAWS CENTER Texas 78539 United States,3114 N ROGERS RD. Edinburg TX 78541
893,Mcallen,2017-01-01,Cat,-98.119870,26.186025,-97.599967,27.849599,118.939245,LA HERENDA ESTATES Texas 78516 United States,4213 NOLFORD PL Corpus Christi TX 78410
896,Mcallen,2017-01-01,Dog,-98.131244,26.191093,-97.378156,27.667156,111.747876,alamo across Walmart Texas 78516 United States,3701 Cimarron blvd. Corpus Christi TX 78414
913,Mcallen,2017-01-01,Dog,-96.338650,33.430090,-98.253652,26.269993,506.375801,Bentsen and Trenton Texas 78501 United States,3921 Swallow Avenue Mcallen TX 78504
982,Mcallen,2017-01-01,Dog,-94.624631,31.471038,-98.159125,26.294325,415.859687,107 AND TOWER RD. Texas 78541 United States,7112 Jodin Drive Edinburg TX 78539
986,Mcallen,2017-01-01,Cat,-98.260979,26.273535,-97.644389,30.462949,290.931617,Toucan Ave Texas 78504 United States,1203 Disraeli Circle Pflugerville TX 78660
997,Mcallen,2017-01-01,Dog,-98.220790,26.267010,-99.333050,31.462733,364.114128,N 11TH ST AND SWALLOW Texas 78504 United States,TX
1043,Mcallen,2017-01-01,Dog,-104.345450,29.551910,-98.130984,26.347624,439.298978,Santa Cruz and Santa Fe Texas 78539 United S...,2111 E mile 19th N Edinburg TX 78542


In [75]:
# save the preprocessed data back into animal data df
animal_data[animal_data['shelter_id'] == 'Mcallen'] = mcallen_df
animal_data

,shelter_id,intake_date,Species,found_lng,found_lat,outcome_lng,outcome_lat,distance_miles,found_address,outcome_address
0,Rochester,2021-01-28,Dog,-77.633991,43.177237,-77.634618,43.178387,0.085445,158 Glenwood ave Rochester 14613,54 Tacoma Street Rochester NY 14613
1,Rochester,2021-01-30,Dog,-77.599173,43.185495,-77.583367,43.171650,1.245466,1094 HUDSON AVE 14621 Rochester 14621,31 CROMBIE ST Rochester NY 14605
2,Rochester,2021-01-30,Dog,-77.617074,43.241926,-77.615373,43.249483,0.528722,17 Hughes pl Rochester 14608,4313 LAKE AVE Rochester NY 14612
3,Rochester,2021-02-01,Dog,-77.643231,43.180933,-77.645970,43.179665,0.163735,DRIVING PK and 2 CANARY ST. Rochester 14608,1 CANARY ST Rochester NY 14613
4,Rochester,2021-02-03,Dog,-77.650684,43.143780,-77.653187,43.145435,0.170467,74 Thurston Rd Rochester 14608,216 DEPEW Street Rochester NY 14611
...,...,...,...,...,...,...,...,...,...,...
23075,Atlanta,2019-01-31,Dog,-84.288835,33.726502,-84.276410,33.706781,1.536071,"2476 Brentwood Road DECATUR, 30032, GA",2176 Loving Drive Decatur GA 30037
23076,Atlanta,2019-11-18,Dog,-84.191581,33.718448,-84.192158,33.718910,0.046022,4822 Truitt Ln GA 30035,4807 Wilkins Station Dr GA 30035
23077,Atlanta,2019-11-19,Dog,-84.280404,33.829848,-84.281226,33.830973,0.090819,"1631 Hartland Drive Decatur, 30033 GA",1668 Hartland Dr Decatur GA 30033
23078,Atlanta,2019-02-12,Dog,-84.285380,33.697093,-84.283554,33.698049,0.124132,2549 Mcglynn Drive GA 30034,254 McGlynn Drive Decatur GA 30034


I then manually checked a few datapoints with the largest distance, there are still errors. In the following example, the address is in Texas, but geopy read it as Wisconsin. This indicates that even though we reduced noise in the addresses to some extent, there are still many problems in the text data that lead to error in calculating the distances. In many cases, using zipcodes instead of full addresses yield more accurate results.

>If we were given more time, further area to reduce noise in address text data would be to use natural language processing packages such as NLTK to analyze text data, get rid of stop words and irrelebant punctuation. 

In [91]:
start = nom.geocode("10th - wisconsin Texas 78501 United States")
start

Location(Texas, Wisconsin, (45.07561000000004, -89.56095999999997, 0.0))

In [84]:
# export new data as excel, without indexes
with pd.ExcelWriter('data_0723.xlsx') as writer:  
    animal_data.to_excel(writer, index = False, sheet_name='animal_data')
    shelter_data.to_excel(writer, index = False, sheet_name='shelter_data')